In [23]:
!pip install transformers datasets seqeval

import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset
import pandas as pd

# Load pre-trained model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=3)




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e282efa968fed57ca4d3e10f7f365ed64402c6da41e03edd66634840054979f3
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
from google.colab import files
uploaded = files.upload()


Saving labeled_data_custom.conll to labeled_data_custom (1).conll


In [26]:
# Display the uploaded files to ensure the file is there
for filename in uploaded.keys():
    print(f'Uploaded file: {filename}')


Uploaded file: labeled_data_custom (1).conll


In [27]:
import pandas as pd

# Get the filename from the uploaded dictionary
filename = list(uploaded.keys())[0]
print(f'Loading file: {filename}')

# Initialize lists to hold tokens and labels
tokens = []
labels = []

# Read the file line by line
with open(filename, 'r') as file:
    for line in file:
        line = line.strip()
        if line:  # Skip empty lines
            parts = line.split(maxsplit=1)  # Split by the first space only
            if len(parts) == 2:  # Ensure there are exactly 2 parts
                token, label = parts
                tokens.append(token)
                labels.append(label)
            else:
                print(f"Skipping line: {line}")  # Debugging output for lines that don't match the format

# Create a DataFrame
df = pd.DataFrame({'Token': tokens, 'Label': labels})

# Display the first few rows of the DataFrame to ensure it was loaded correctly
print(df.head(10))


Loading file: labeled_data_custom (1).conll
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skipping line: O
Skip

In [28]:
# Check unique labels
unique_labels = df['Label'].unique()
print(f"Unique labels: {unique_labels}")


Unique labels: ['O' 'I-Product' 'B-Product' 'I-PRICE' 'I-LOC' 'B-LOC' 'B-PRICE']


In [29]:
# Create a label mapping
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print(f"Label mapping: {label_mapping}")


Label mapping: {'O': 0, 'I-Product': 1, 'B-Product': 2, 'I-PRICE': 3, 'I-LOC': 4, 'B-LOC': 5, 'B-PRICE': 6}


In [30]:
# Map the labels in the DataFrame
df['Label'] = df['Label'].map(label_mapping)

# Check for any NaN values after mapping
print("Check for NaN values after mapping:")
print(df[df['Label'].isna()])

# Drop any rows with NaN values after mapping
df.dropna(inplace=True)

# Convert labels to integer type
df['Label'] = df['Label'].astype(int)

# Display the DataFrame after mapping
print(df.head(10))


Check for NaN values after mapping:
Empty DataFrame
Columns: [Token, Label]
Index: []
                                 Token  Label
0                                    💥      0
1                                    💥      0
2  ...................................      0
3                                    💥      0
4                                    💥      0
5                                    📌      1
6                          Replacement      1
7                              Silicon      2
8                               Nipple      1
9                                    👍      0


In [ ]:
!pip install datasets


In [31]:
from transformers import AutoTokenizer
import torch

# Load pre-trained model and tokenizer
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['Token'].tolist(),
        padding=True,
        truncation=True,
        is_split_into_words=True,
    )

    labels = []
    word_ids = tokenized_inputs.word_ids(batch_index=0)  # Assuming batch_size is 1
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
        if word_idx is None:
            label_ids.append(-100)
        elif word_idx != previous_word_idx:
            label_ids.append(int(examples['Label'].values[word_idx]))
        else:
            label_ids.append(-100)
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Apply the function to each sentence
tokenized_inputs_list = [tokenize_and_align_labels(sentence) for _, sentence in df.groupby(df.index // 10)]  # Adjust this logic according to your sentence structure

# Convert lists to PyTorch tensors
input_ids = [torch.tensor(item['input_ids'], dtype=torch.long) for item in tokenized_inputs_list]
attention_mask = [torch.tensor(item['attention_mask'], dtype=torch.long) for item in tokenized_inputs_list]
labels = [torch.tensor(item['labels'], dtype=torch.long) for item in tokenized_inputs_list]

# Pad sequences to ensure they have the same length
input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

# Combine into a dataset
from datasets import Dataset

dataset = Dataset.from_dict({
    'input_ids': input_ids,
    'attention_mask': attention_mask,
    'labels': labels
})

print(dataset)


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 331
})


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load the model
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_mapping))

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",  # Change to eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
)

trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Save the model
model.save_pretrained('./fine-tuned-ner-model')
tokenizer.save_pretrained('./fine-tuned-ner-model')


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
